## Determine ADCP start/stop times from cruise logs (Endeavor)

In [1]:
import os

# your paths may vary
ar28_log_path = r'\\sosiknas1\Lab_data\LTER\20180404_AR28\adcp\raw\log\DAS_main.log'
ar31_log_path = r'\\sosiknas1\Lab_data\LTER\20181020_AR31\adcp\raw\log\DAS_main.log'
en617_log_path = r'\\sosiknas1\Lab_data\LTER\20180720_EN617\adcp\raw\log\DAS_main.log'
en608_log_path = r'\\sosiknas1\Lab_data\LTER\20180131_EN608\adcp\raw\log\DAS_main.log'

log_path = ar28_log_path
assert os.path.exists(log_path)

Read logfile

In [2]:
with open(log_path) as fin:
    lines = fin.readlines()

# drop all lines with a leading space, those represent parameters
lines = list(filter(lambda l: not l.startswith(' '), lines))
lines[:5]

['2018-01-26 20:39:29,669 INFO     cruisesetup  StartCruise, new, cruiseid is ar27\n',
 '2018-01-26 20:39:33,024 INFO     root         entering StartLogging, resume = 0, auto = False\n',
 '2018-01-26 20:39:33,037 INFO     root         Commands for wh300:\n',
 '2018-01-26 20:39:41,918 INFO     root         Commands for os150:\n',
 '2018-01-26 20:39:41,919 INFO     root         Commands for os38:\n']

In [3]:
# convert into Pandas dataframe

import re
import pandas as pd
from io import StringIO

ssv = ''.join(lines)

df = pd.read_csv(StringIO(ssv), delimiter=r'\s\s+', header=None, engine='python')
df.columns = ['datetime', 'code', 'message']
df.head()

,datetime,code,message
0,"2018-01-26 20:39:29,669 INFO",cruisesetup,"StartCruise, new, cruiseid is ar27"
1,"2018-01-26 20:39:33,024 INFO",root,"entering StartLogging, resume = 0, auto = False"
2,"2018-01-26 20:39:33,037 INFO",root,Commands for wh300:
3,"2018-01-26 20:39:41,918 INFO",root,Commands for os150:
4,"2018-01-26 20:39:41,919 INFO",root,Commands for os38:


In [4]:
# convert datetime column into datetime64 type

df.datetime = df.datetime.str.replace(' INFO','')
df.datetime = pd.to_datetime(df.datetime)
df.pop('code')
df.head()

,datetime,message
0,2018-01-26 20:39:29.669,"StartCruise, new, cruiseid is ar27"
1,2018-01-26 20:39:33.024,"entering StartLogging, resume = 0, auto = False"
2,2018-01-26 20:39:33.037,Commands for wh300:
3,2018-01-26 20:39:41.918,Commands for os150:
4,2018-01-26 20:39:41.919,Commands for os38:


Find logging start/stop messages

In [5]:
df[df.message.str.match('Logging (started|stopped)')].tail(20)

,datetime,message
201,2018-03-29 02:00:14.450,Logging started
206,2018-03-30 11:08:14.287,Logging stopped
211,2018-03-30 11:22:39.133,Logging started
214,2018-03-30 15:45:36.390,Logging stopped
219,2018-03-30 15:52:06.626,Logging started
222,2018-03-30 16:08:49.282,Logging stopped
228,2018-03-30 16:09:32.678,Logging started
231,2018-03-31 14:54:28.663,Logging stopped
236,2018-03-31 14:55:26.679,Logging started
239,2018-03-31 18:17:42.086,Logging stopped


Sometimes these files have multiple cruises in them, start/end cruise events delimit that

In [6]:
df[df.message.str.match('.*(Start|End)Cruise')]

,datetime,message
0,2018-01-26 20:39:29.669,"StartCruise, new, cruiseid is ar27"
25,2018-02-26 13:16:50.228,"EndCruise, cruiseid is ar27"
49,2018-02-26 13:18:06.596,"StartCruise, new, cruiseid is ar27_halfmbins"
58,2018-02-26 13:36:24.766,"EndCruise, cruiseid is ar27_halfmbins"
70,2018-02-26 13:36:52.736,"StartCruise, old, cruiseid is ar27_halfmbins"
109,2018-03-05 22:01:27.506,"EndCruise, cruiseid is ar27_halfmbins"
120,2018-03-05 22:07:03.960,"StartCruise, new, cruiseid is plots_testing"
129,2018-03-05 22:08:54.260,"EndCruise, cruiseid is plots_testing"
140,2018-03-23 13:27:55.299,"StartCruise, new, cruiseid is ar28a"
248,2018-04-01 13:27:20.700,"EndCruise, cruiseid is ar28a"
